In [ ]:
import requests
import json
from utils import AWANLLM_API_KEY, MODEL


url = "https://api.awanllm.com/v1/chat/completions"

payload = json.dumps({
  "model": MODEL,
  "messages": [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Hello!"},
    {"role": "assistant", "content": "Hi!, how can I help you today?"},
    {"role": "user", "content": "Can you write me Haiku about Hare and Tortoise"}
  ],
  "repetition_penalty": 1.1,
  "temperature": 0.7,
  "top_p": 0.9,
  "top_k": 40,
  "max_tokens": 1024,
  "stream": True
})
headers = {
  'Content-Type': 'application/json',
  'Authorization': f"Bearer {AWANLLM_API_KEY}"
}

response = requests.request("POST", url, headers=headers, data=payload)

print(response.text)



In [5]:
# Collect the streamed response into a full sentence

full_response = ""

# Iterate through the streaming data
for line in response.iter_lines():
    if line:  # Avoid processing empty lines
        decoded_line = line.decode('utf-8').strip()  # Strip leading/trailing spaces
        
        if decoded_line == "data: [DONE]":
            break
        
        if decoded_line.startswith("data: "):
            try:
                # Extract the JSON part after 'data:'
                json_data = json.loads(decoded_line.split("data: ")[1])
                # Accumulate the text parts from the response
                full_response += json_data["choices"][0]["delta"]['content']
            except (IndexError, json.JSONDecodeError) as e:
                # Print the problematic line for debugging
                print(f"Error decoding JSON from line: {decoded_line}")
                print(f"Exception: {e}")

print("Full response:", full_response)

Full response: Here is a haiku:

Slow and steady wins
Tortoise smiles as hare sleeps on
Patience is key


In [8]:
from openai import OpenAI 
import os
import json
import time
from dotenv import load_dotenv
import prompt_design
from openai_api import OpenAIClient
load_dotenv()

API_KEY = os.getenv("OPENAI_API_KEY")
MODEL = os.getenv("OPENAI_MODEL")


openai = OpenAIClient()
system_prompt = prompt_design.PROMPT_TEMPLATE

# user_messages = [
#     'Giới thiệu cho tôi các sản phẩm bán chạy nhất.',
#     'Ở đây có bán gì rẻ rẻ không?',
#     'Cây son mắc nhất trên hasaki là cây nào?'
# ]

# for message in user_messages:
#     response = client.get_response(user_message=message, prompt=system_prompt)
#     print(f"User: {message}\nAssistant: {response}")
 
# Test `get_json_from_prompt`
print("\n=== Testing `get_json_from_prompt` ===")
guide = '''
    Phân tích yêu cầu người dùng theo nhu cầu sau và trả kết quả tương ứng như mô tả. Không thêm bớt hay tự chế ra kết quả.
    thay thế cụm có dấu <>, không kèm dấu này, câu trả lời theo dạng json format.
    (0) Nếu người dùng chỉ hỏi một câu chào hỏi lịch sự bình thường, hãy trả lại với cú pháp: {'signal':'0', 'query':'<câu hỏi>'}
    (1) Nếu người dùng nhắc đến việc tìm hiểu một sản phẩm rất cụ thể, hãy trả lời với cú pháp: {'signal':'1','product_term':'<tên sản phẩm>'}
    (2) Nếu người dùng nhắc đến việc so sánh hai sản phẩm rất cụ thể, hãy trả lời với cú pháp: {'signal':'2','product_term_1'='<thông tin sản phẩm 1>','product_term_2'='<thông tin sản phẩm 2>'}
    (3) Nếu người dùng nhắc đến việc tìm sản phẩm theo yêu cầu:
        hãy trả về các thuộc tính có trong cú pháp sau (nếu có), trong đó signal và product_term là bắt buộc: {'signal':'3','product_term':'<thông tin sản phẩm>', 'product_ingredients': '<thành phần sản phẩm>','price':'<price>', 'operator_price':'<operator>', 'rating':'<rating>', 'operator_rating':'<operator>', 'others':'<thông tin khác>'}
        Lưu ý <operator> sẽ là các operator trong mongodb như '$gte', '$lte' khi câu hỏi liên quan đến giá lớn hơn, bé hơn bao nhiêu đó. Hoặc rating cao hơn hay thấp hơn bao nhiêu đó.
        Và <price> phải được chuyển về theo dạng ví dụ 10k là 10000.
    (4) Nếu người dùng cần hỗ trợ về các chính sách mua hàng, giao hàng, đổi trả, hỗ trợ kỹ thuật, hãy trả lời với cú pháp: {'signal':'4','query':'<query>'}
    (5) Nếu người dùng cung cấp một số tiền và cần hỗ trợ mua hàng dựa trên số tiền hiện có, hãy trả lời với cú pháp: 
    {'signal':'5','budget':'<budget>','product_term':'<Danh sách các term về sản phẩm>'}; 
    chú ý, budget phải là một con số hoặc một string có thể chuyển thành int một cách trực tiếp.
    product term ở đây là một danh sách các sản phẩm gồm sản phẩm đầu là sản phẩm chính và các sản phẩm sau là sản phẩm mà người dùng có thể cũng mua chung được cách nhau bởi dấu phẩy
    
    Nếu yêu cầu không cụ thể về sản phẩm nào, trả lại theo cú pháp: {'signal':'3','product_term':'<mô tả sản phẩm>','others':'<thông tin khác>'}

    '''

completion = openai.client.chat.completions.create(
    model=MODEL,
    messages= [
        { 'role': "system", 'content':guide },
        {'role': "user", 'content': "gợi ý cho tôi sản phẩm trang điểm dưới 200k"}
        ],
    response_format={"type": "json_object"}
    )
# response.choices[0].message
assistant_response = completion.choices[0].message.content
print("JSON Response:", assistant_response)


=== Testing `get_json_from_prompt` ===
JSON Response: {
  "signal": "5",
  "budget": "200000",
  "product_term": "son môi, phấn nền, phấn mắt"
}
